In [1]:
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.", # -> [오늘, 날씨가, 좋아서 ...]
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

### 자연어 전처리

In [2]:
from mecab import MeCab
mecab = MeCab()
# 형태분석기 사용해 명사만 추출
morphs_list = list()
for sentence in sentences :
    morphs = mecab.nouns(sentence)
    morphs_list = morphs_list + morphs
morphs_list

['날씨',
 '나',
 '영화',
 '음식',
 '운동',
 '건강',
 '것',
 '공부',
 '여행',
 '계획',
 '어디',
 '책',
 '마음',
 '편안',
 '오늘',
 '친구',
 '언어',
 '것',
 '흥미',
 '주말',
 '가족',
 '시간']

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True)
features = vectorizer.fit_transform(morphs_list)  # features 생성

### LDA 적용

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=111)

In [5]:
lda_model.fit(features)

LatentDirichletAllocation(n_components=3, random_state=111)

In [6]:
# lda_model.components_
# len(lda_model.components_[0])
topics_list = lda_model.components_[0]  # 첫번째 토픽
topics_list  # 확인

array([0.33654237, 0.33714206, 0.33654245, 1.32934725, 0.33714184,
       0.33654233, 1.32934644, 0.33714199, 0.33654233, 0.33714199,
       0.33714194, 1.32934726, 0.33714188, 1.32934726, 0.33654235,
       0.33714198, 0.33714192, 0.33654237])

In [7]:
#topics_list = lda_model.components_
lda_model.components_

array([[0.33654237, 0.33714206, 0.33654245, 1.32934725, 0.33714184,
        0.33654233, 1.32934644, 0.33714199, 0.33654233, 0.33714199,
        0.33714194, 1.32934726, 0.33714188, 1.32934726, 0.33654235,
        0.33714198, 0.33714192, 0.33654237],
       [0.33556197, 1.32641014, 0.33556199, 0.33516318, 1.32641046,
        0.33556197, 0.33516348, 1.32641023, 0.33556196, 1.32641026,
        1.32641033, 0.33516318, 1.32641041, 0.33516318, 0.33556197,
        1.32641028, 1.32641037, 0.33556197],
       [1.32789566, 0.33644779, 1.32789556, 0.33548956, 0.3364477 ,
        1.3278957 , 0.33549008, 0.33644778, 1.32789571, 0.33644775,
        0.33644774, 0.33548956, 0.33644771, 0.33548956, 1.32789568,
        0.33644774, 0.33644771, 1.32789566]])

### 토픽별 단어 표시

In [8]:
dictionary_list = vectorizer.get_feature_names_out()
dictionary_list
# len(vectorizer.get_feature_names_out())

array(['가족', '건강', '계획', '공부', '날씨', '마음', '시간', '어디', '언어', '여행', '영화',
       '오늘', '운동', '음식', '주말', '친구', '편안', '흥미'], dtype=object)

In [9]:
import pandas as pd
df_datas = [topics_list, dictionary_list]   #2차원 배열로 넣어 순서대로 생성시킴
df_topics = pd.DataFrame(data=df_datas)
df_topics = df_topics.T   # T(transport) : row를 컬럼으로 만들어줌 
df_topics[:2]

,0,1
0,0.336542,가족
1,0.337142,건강


In [10]:
df_topics.columns

RangeIndex(start=0, stop=2, step=1)

In [11]:
df_topics.sort_values(0, ascending=False)  

,0,1
13,1.329347,음식
11,1.329347,오늘
3,1.329347,공부
6,1.329346,시간
1,0.337142,건강
7,0.337142,어디
9,0.337142,여행
15,0.337142,친구
10,0.337142,영화
16,0.337142,편안


In [20]:
import numpy as np

### 댓글과 주요 토픽 연결

In [15]:
# 상위 단어 추출
for topic in lda_model.components_ :
    df_datas = [topic, dictionary_list]   
    df_topics = pd.DataFrame(data=df_datas)
    df_topics = df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)  
    # print(df_topics[:3])   
    topics_text = ' '.join(df_topics[1].values[:4])
    print(topics_text)   # 각 topic 마다 해당하는 것 출력됨
    topics_list.append(topics_text)

음식 오늘 공부 시간


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [16]:
df_topics[1].values[:4]    # get values from Serise , list형식으로 잡힘 , 순서대로 출력됨

array(['음식', '오늘', '공부', '시간'], dtype=object)

In [17]:
' '.join(df_topics[1].values[:4])

'음식 오늘 공부 시간'

In [18]:
topics_list_add = [topics_list,['Topic0','Topic1','Topic2']]
df_topic_keywords = pd.DataFrame(topics_list_add)
df_topic_keywords.T

,0,1
0,0.336542,Topic0
1,0.337142,Topic1
2,0.336542,Topic2
3,1.329347,NaN
4,0.337142,NaN
5,0.336542,NaN
6,1.329346,NaN
7,0.337142,NaN
8,0.336542,NaN
9,0.337142,NaN


In [19]:
topics_output = lda_model.transform(features)
topics_output

array([[0.16857425, 0.66320011, 0.16822564],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16857425, 0.66320011, 0.16822564],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822564],
       [0.16857425, 0.6632001 , 0.16822565],
       [0.33333333, 0.33333333, 0.33333333],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822565],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16857425, 0.66320011, 0.16822565],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16857425, 0.66320011, 0.16822564],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822564],
       [0.16827277, 0.16778152, 0.66394571],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16827277, 0.16778152, 0.66394571],
       [0.66467305, 0.16758181, 0.16774514]])

## 각 댓글별 topics 분류

In [21]:
lda_model.n_components

3

In [26]:
df_topics_score = pd.DataFrame(topics_output)  # 각 댓글 당 토픽 분류 점수

In [31]:
# 각 댓글 마다 토픽 분류 지정
import numpy as np
dominat_in_topic = np.argmax(topics_output, axis=1)

In [ ]:
df_topics_score